In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.stats import norm

## Task 1

$y_n = \theta + e_n$

In [ ]:
theta = np.random.normal(loc=0, scale=1, size=1)[0]
theta

In [ ]:
ssq_e = 1
m_e = 0
N = 100
e = np.random.normal(loc=m_e, scale=np.sqrt(ssq_e), size=N)
y1 = theta*np.ones(N) + e

In [ ]:
plt.figure()
plt.plot(np.arange(N), y1)
plt.xlabel('n')
plt.ylabel('y')
plt.title('theta: {}'.format(round(theta, 4)))

In [ ]:
theta_ML = np.sum(y1)/N
theta_ML

Prior parameters

In [ ]:
m_th = 0
ssq_th = 1

In [ ]:
theta_MAP = (m_th*ssq_e/ssq_th + np.sum(y1))/(N + ssq_e/ssq_th)
ssq_post = ssq_e / (N + ssq_e/ssq_th)
theta_MAP

In [ ]:
axis = np.linspace(-5, 5, num=10000)
plt.figure()
plt.plot(axis, norm.pdf(axis, loc=theta_ML, scale=np.sqrt(ssq_e/N)), label='Likelihood')
plt.plot(axis, norm.pdf(axis, loc=m_th, scale=np.sqrt(ssq_th)), label='Prior')
plt.plot(axis, norm.pdf(axis, loc=theta_MAP, scale=np.sqrt(ssq_post)), label='Posterior')
plt.xlabel('theta')
plt.title('N: {}, theta: {}, prior mean: {}, prior variance: {}, noise variance: {}'.format(N, round(theta, 4), round(m_th, 4), round(ssq_th, 4), round(ssq_e, 4)))
plt.legend()

## Task 2

$y_n = \theta_1 + \theta_2n + e_n$

In [ ]:
theta1 = np.random.normal(loc=0, scale=1, size=1)[0]
theta2 = np.random.normal(loc=0, scale=1, size=1)[0]
y2 = theta1 + theta2*np.arange(N) + e

In [ ]:
plt.figure()
plt.plot(np.arange(N), y2)
plt.xlabel('n')
plt.ylabel('y')
plt.title('theta1: {}, theta2: {}'.format(round(theta1, 4), round(theta2, 4)))

In [ ]:
def polynomial_G(N_samps, N_params):
    # returns generator matrix G for a polynomial model with P coefficients and N observations
    G = np.zeros((N_samps, N_params))
    for i in range(N_params):
        G[:,i] = np.power(np.arange(N_samps), i)
    return G

In [ ]:
def get_theta_ML(x, G):
    return np.matmul(np.linalg.inv(np.matmul(G.T, G)), np.matmul(G.T, x))

In [ ]:
G2 = polynomial_G(N, 2)

In [ ]:
get_theta_ML(y2, G2)

In [ ]:
def get_posterior(x, G, prior_mean, C_inv, noise_variance):
    # terms as in handout
    phi = np.matmul(G.T, G) + noise_variance * C_inv
    Theta = np.matmul(G.T, x) + noise_variance * np.matmul(C_inv, prior_mean)
    mean = np.matmul(np.linalg.inv(phi), Theta)
    covar = noise_variance * np.linalg.inv(phi)
    
    return mean, covar

Prior parameters

In [ ]:
C_th2 = np.eye(2)
m_th2 = np.zeros(2)

In [ ]:
m_post, C_post = get_posterior(y2, G2, m_th2, np.linalg.inv(C_th2), ssq_e)
m_post, C_post

In [ ]:
n_tilde = G2
m_star = np.matmul(n_tilde, m_post) + m_e
v_star = np.zeros(n_tilde.shape[0])
for i in range(v_star.shape[0]):
    v_star[i] = np.linalg.multi_dot((n_tilde[i], C_post, n_tilde[i])) + ssq_e

In [ ]:
fig, ax, = plt.subplots()
ax.plot(np.arange(N), m_star)
ax.plot(np.arange(N), y2)
plt.xlabel('n')
plt.ylabel('y')
plt.title('Example 3')
ax.fill_between(np.arange(N), (m_star-3*np.sqrt(v_star)), (m_star+3*np.sqrt(v_star)), color='b', alpha=.1)

## Task 3

Posterior model probabilites:

In [ ]:
#yt = np.random.normal(loc=0, scale=np.sqrt(ssq_e), size=N)
yt = np.random.normal(loc=0, scale=1, size=1)[0]*np.ones(N) + np.random.normal(loc=0,scale=np.sqrt(ssq_e), size=N)
#yt = np.random.normal(loc=0, scale=1, size=1)[0]*np.arange(N) + np.random.normal(loc=0, scale=1, size=1)[0]*np.ones(N) + np.random.normal(loc=0,scale=np.sqrt(ssq_e), size=N)

In [ ]:
plt.figure()
plt.plot(np.arange(N), yt)
plt.title('Model 3')
plt.xlabel('n')
plt.ylabel('y')

 $\mathcal{M}_1$

In [ ]:
def get_lpM1(data, noise_variance):
    Num_samps = data.shape[0]
    return -Num_samps*np.log(2*np.pi*noise_variance)/2 - np.sum(np.square(data))/(2*noise_variance)

In [ ]:
lpM1 = get_lpM1(yt, ssq_e)
lpM1

 $\mathcal{M}_2$

In [ ]:
def get_lpM2(data, noise_variance, prior_mean, prior_variance):
    Num_samps = data.shape[0]
    t1 = -Num_samps*np.log(2*np.pi)/2
    t2 = -np.log(Num_samps*prior_variance + noise_variance)/2
    t3 = -(Num_samps-1)*np.log(noise_variance)/2
    t4 = np.sum(np.square(data))
    t5 = noise_variance*np.square(prior_mean) / prior_variance
    t6 = np.square(np.sum(data) + noise_variance*prior_mean / prior_variance) / (Num_samps + noise_variance/prior_variance)
    
    return t1+t2+t3 - (t4+t5-t6)/(2*noise_variance)

In [ ]:
lpM2 = get_lpM2(yt, ssq_e, 0, 100000000000)
lpM2

 $\mathcal{M}_3$

In [ ]:
def get_lpM3(data, noise_variance, prior_mean, C_inv, Gen):
    Num_samps = data.shape[0]
    phi = np.matmul(Gen.T, Gen) + noise_variance * C_inv
    Theta = np.matmul(Gen.T, data) + noise_variance * np.matmul(C_inv, prior_mean)
    th_map = np.matmul(np.linalg.inv(phi), Theta)
    sign_C, log_det_C_inv = np.linalg.slogdet(C_inv)
    sign_P, log_det_phi = np.linalg.slogdet(phi)
    
    t1 = -Num_samps*np.log(2*np.pi)/2
    t2 = sign_C*log_det_C_inv/2
    t3 = -sign_P*log_det_phi/2
    t4 = -(Num_samps-2)*np.log(noise_variance)/2
    
    t5 = np.sum(np.square(data))
    t6 = noise_variance * np.linalg.multi_dot((prior_mean, C_inv, prior_mean))
    t7 = np.matmul(Theta.T, th_map)
    
    
    return t1+t2+t3+t4 - (t5+t6-t7)/(2*noise_variance)

In [ ]:
lpM3 = get_lpM3(yt, ssq_e, np.zeros(2), 100000000000000*np.eye(2), G2)
lpM3

## Task 4

In [ ]:
N_n = 105 # easier if we generate longer sample to deal with case where offset > 85, then truncate
N_f = 100
sigma_n = 2
sigma_gamma = 1
signal = np.array([-3, 5, -2, 4, 1, 3, 5, -2, 2, 4, 6, 5, -2, -2, 1]).astype(np.float64)
noise = sigma_n*np.random.normal(size=N_n)
gamma = sigma_gamma*np.random.normal(size=1)[0].astype(np.float64)
offset = np.round(90*np.random.rand()).astype(np.int16)
print(gamma)
print(offset)
y3 = noise
signal_offset = np.zeros(N_n)
signal_offset[offset:offset+15] = gamma * signal

y4 = (y3 + signal_offset)[:N_f]

In [ ]:
ssq_gam_prior = np.square(sigma_gamma)
mu_gam_prior = -gamma

In [ ]:
plt.figure()
plt.plot(np.arange(N_f), y4)
plt.xlabel('n')
plt.ylabel('y')
plt.title('Hidden signal. Offset: {}, Gamma: {}, Noise Variance: {}'.format(offset, round(gamma, 3), np.square(sigma_n)))
          

In [ ]:
def get_gamma_ML(data, lamb, sig_vec):
    Num_samps = data.shape[0]
    signal_shift = np.pad(sig_vec, (lamb, max(Num_samps-lamb-15, 0)))[:Num_samps]
    numer = np.sum(signal_shift*data)
    denom = np.sum(np.square(sig_vec))
    return numer/denom

In [ ]:
get_gamma_ML(y4, offset, signal)

In [ ]:
def get_log_model_evidence(data, lamb, sig_vec, noise_variance, prior_mean, prior_variance):
    Num_samps = data.shape[0]
    signal_shift = np.pad(sig_vec, (lamb, max(Num_samps-lamb-15, 0)))[:Num_samps]
    
    t1 = -Num_samps*np.log(2*np.pi)/2
    t2 = -(Num_samps-1)*np.log(noise_variance)/2
    t3 = -np.log(prior_variance*(np.sum(np.square(sig_vec))+noise_variance))/2
    
    t4 = np.sum(np.square(data))
    t5 = noise_variance*np.square(prior_mean)/prior_variance
    t6 = noise_variance*prior_mean/prior_variance + np.sum(data*signal_shift)
    t7 = np.sum(np.square(sig_vec)) + noise_variance/prior_variance
    
    return t1+t2+t3 - (t4+t5-(np.square(t6))/t7)/(2*noise_variance)

In [ ]:
get_log_model_evidence(y4, offset, signal, np.square(sigma_n), mu_gam_prior, ssq_gam_prior)

In [ ]:
evidences = np.zeros(91)

In [ ]:
for i in range(91):
    evidences[i] = get_log_model_evidence(y4, i, signal, np.square(sigma_n), mu_gam_prior, ssq_gam_prior)

In [ ]:
offset_est = np.argmax(evidences)
offset_est

In [ ]:
get_log_model_evidence(y4, offset_est, signal, np.square(sigma_n), 0, np.square(sigma_gamma))

In [ ]:
plt.figure()
plt.plot(np.arange(91), evidences)
plt.title('True offset: {}, Estimated offset: {}, Gamma: {}, Noise Variance: {}'.format(offset, offset_est, round(gamma, 3), np.square(sigma_n)))
plt.xlabel('lambda')
plt.ylabel('log model evidence')

In [ ]:
get_gamma_ML(y4, offset_est, signal)

In [ ]:
def get_log_null_hypothesis(data, noise_variance):
    Num_samps = data.shape[0]
    return -Num_samps*np.log(2*np.pi*noise_variance)/2 - np.sum(np.square(data))/(2*noise_variance)

In [ ]:
get_log_null_hypothesis(y4, sigma_n)

In [ ]:
def get_gamma_posterior(data, lamb, sig_vec, noise_variance, prior_mean, prior_variance):
    Num_samps = data.shape[0]
    signal_shift = np.pad(sig_vec, (lamb, max(Num_samps-lamb-15, 0)))[:Num_samps]
    
    pos_mea = (noise_variance*prior_mean/prior_variance + np.sum(data*signal_shift))/(np.sum(np.square(sig_vec)) + noise_variance/prior_variance)
    pos_var = (noise_variance)/(np.sum(np.square(sig_vec)) + noise_variance/prior_variance)
    return pos_mea, pos_var


def get_gamma_likelihood(data, lamb, sig_vec, noise_variance):
    Num_samps = data.shape[0]
    signal_shift = np.pad(sig_vec, (lamb, max(Num_samps-lamb-15, 0)))[:Num_samps]
    
    lik_mea = np.sum(data*signal_shift) / np.sum(np.square(sig_vec))
    lik_var = noise_variance / np.sum(np.square(sig_vec))
    return lik_mea, lik_var

In [ ]:
#ssq_gam_prior = np.square(sigma_gamma)
mu_gam_like, ssq_gam_like = get_gamma_likelihood(y4, offset_est, signal, np.square(sigma_n))
mu_gam_post, ssq_gam_post = get_gamma_posterior(y4, offset_est, signal, np.square(sigma_n), mu_gam_prior, ssq_gam_prior)
mu_gam_post

In [ ]:
axis2 = np.linspace(-5, 5, num=10000)
plt.figure()
plt.plot(axis2, norm.pdf(axis2, loc=mu_gam_like, scale=np.sqrt(ssq_gam_like)), label='Likelihood')
plt.plot(axis2, norm.pdf(axis2, loc=mu_gam_prior, scale=np.sqrt(ssq_gam_prior)), label='Prior')
plt.plot(axis2, norm.pdf(axis2, loc=mu_gam_post, scale=np.sqrt(ssq_gam_post)), label='Posterior')
plt.legend()
plt.xlabel('gamma')